<a href="https://colab.research.google.com/github/visumania/DeepSexist/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Montura de google colab para poder leer/escribir en Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importación de librerías necesarias

In [2]:
import pandas as pd

Lectura y tratamiento de los datos del fichero

In [4]:
df = pd.read_json('/content/drive/MyDrive/EXIST 2025 Videos Dataset/training/EXIST2025_training.json', orient='index') # Lectura del dataset desde Drive
df.reset_index(drop=True, inplace=True) # Reseteamos el index del dataframe
df = df.drop(columns=["id_Tiktok", "id_EXIST", "video", "url", "annotators", "gender_annotators", "labels_task3_2", "split"]) # Eliminación de columnas que no son relevantes del dataframe

In [5]:
# Distribución de datos importantes en el fichero

print("Distribución del número de anotadores")
print(df.value_counts("number_annotators"))

print("\nDistribución del idioma")
print(df.value_counts("lang"))

print("\nDistribución del etiquetado para la tarea 1")
print(df['labels_task3_1'].value_counts())

print("\nDistribución del etiquetado para la tarea 3")
print(df['labels_task3_3'].value_counts())

Distribución del número de anotadores
number_annotators
2    1780
3     744
Name: count, dtype: int64

Distribución del idioma
lang
es    1524
en    1000
Name: count, dtype: int64

Distribución del etiquetado para la tarea 1
labels_task3_1
[NO, NO]               1069
[YES, YES]              711
[YES, NO, YES]          255
[NO, YES, NO]           140
[YES, YES, YES]         108
[NO, YES, YES]          101
[YES, NO, NO]            62
[UNKNOWN, NO, NO]        20
[UNKNOWN, YES, YES]      19
[NO, UNKNOWN, NO]        15
[UNKNOWN, YES, NO]        7
[UNKNOWN, NO, YES]        6
[YES, UNKNOWN, YES]       4
[YES, YES, NO]            4
[YES, UNKNOWN, NO]        2
[NO, UNKNOWN, YES]        1
Name: count, dtype: int64

Distribución del etiquetado para la tarea 3
labels_task3_3
[[-], [-]]                                                                                                               1069
[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMINANCE]]                                                

Tratamiento del fichero orifinal para generar uno nuevo en donde solamente se recoja la información relevante para la tarea 3.1

La primera tarea es una clasificación binaria. Los sistemas tienen que decidir si un vídeo dado contiene o no expresiones o comportamientos sexistas (es decir, es sexista en sí mismo, describe una situación sexista o critica un comportamiento sexista), y clasificarlos según dos categorías: **YES** y **NO**

In [6]:
# Eliminamos la columna correspondiente a la tarea 3.3
df_3_1 = df.drop(columns=["labels_task3_3"])

# Expande la columna labels_task3_1 en varias columnas
df_3_1_expanded = df_3_1["labels_task3_1"].apply(pd.Series)

# Renombra las columnas resultantes para mayor claridad
df_3_1_expanded.columns = [f"labels_task_3_1_{i+1}" for i in range(df_3_1_expanded.shape[1])]

# Une con el dataframe original sin la columna original
df_3_1 = pd.concat([df_3_1.drop(columns=["labels_task3_1"]), df_3_1_expanded], axis=1)

# Pasamos a binario los valores YES y NO
df_3_1.iloc[:, 4:] = df_3_1.iloc[:, 4:].replace({"YES": int(1), "NO": int(0)})

# # Esto se hace si queremos determinar si un valor específico de una fila/columna de un dataframe es NaN
# # print(pd.isna(df_3_1.at[0, 'labels_task_3_1_3']))

df_3_1.iloc[:, 6] = df_3_1.iloc[:, 6].astype("Int64")  # Pasamos los valores binarios de la última columna a int64 --> Tipo entero que soporta NaN

# print(str(type(df_3_1.iloc[0,6]))=="<class 'pandas._libs.missing.NAType'>")

print(df_3_1['labels_task_3_1_1'].value_counts())
print(df_3_1['labels_task_3_1_2'].value_counts())
print(df_3_1['labels_task_3_1_3'].value_counts())

# df_3_1

labels_task_3_1_1
0          1326
1          1146
UNKNOWN      52
Name: count, dtype: int64
labels_task_3_1_2
0          1412
1          1090
UNKNOWN      22
Name: count, dtype: int64
labels_task_3_1_3
1    494
0    250
Name: count, dtype: int64


<ipython-input-6-7a2f2c4d8e45>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_3_1.iloc[:, 4:] = df_3_1.iloc[:, 4:].replace({"YES": int(1), "NO": int(0)})


Unificado de las anotaciones de los diferentes anotadores/as por mayoría: Si tiene 3 anotadores se unifica en el etiquetado que haya predominado en cada anotador/a.
Si hay 2 anotadores/as y conflictos habrá que decidir si dejarlo como 0 o 1

In [7]:
df_3_1["label_task_3_1_merged"] = 0 # Creación de una nueva columna en donde recogeremos la unión de la etiquetas impuestas por los/las anotadores/as

In [8]:
print(str(df_3_1.iloc[1]['labels_task_3_1_3']) == '<NA>')
print(df_3_1.iloc[0]['labels_task_3_1_1'] == 'UNKNOWN')

True
False


In [9]:
for i in range(df_3_1.shape[0]):
  votacionPositiva = votacionNegativa = 0
  numeroAnotadores = df_3_1.iloc[i]['number_annotators'] # Asimilamos que el número de anotadores es el que nos dice la columna pero lo verificaremos en las siguientes filas de código

  if df_3_1.iloc[i]['labels_task_3_1_1'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_1'] == 1:
    votacionPositiva += 1
  elif df_3_1.iloc[i]['labels_task_3_1_1'] == 'UNKNOWN': # En caso de que el etiquetado sea UNKNOWN (contabiliza al anotador pero no nos aporta información por lo que podemos descartarlo)
    numeroAnotadores -= 1

  if df_3_1.iloc[i]['labels_task_3_1_2'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_2'] == 1:
    votacionPositiva += 1
  elif df_3_1.iloc[i]['labels_task_3_1_2'] == 'UNKNOWN': # En caso de que el etiquetado sea UNKNOWN (contabiliza al anotador pero no nos aporta información por lo que podemos descartarlo)
    numeroAnotadores -= 1

  if str(df_3_1.iloc[i]['labels_task_3_1_3']) == '0':
    votacionNegativa += 1
  elif str(df_3_1.iloc[i]['labels_task_3_1_3']) == '1':
    votacionPositiva += 1

  if numeroAnotadores == 3:
    if votacionPositiva > votacionNegativa:
      df_3_1.at[i, 'label_task_3_1_merged'] = 1
    elif votacionPositiva < votacionNegativa:
      df_3_1.at[i, 'label_task_3_1_merged'] = 0

  if numeroAnotadores == 2:
    if votacionPositiva == 2:
      df_3_1.at[i, 'label_task_3_1_merged'] = 1
    elif votacionNegativa == 2:
      df_3_1.at[i, 'label_task_3_1_merged'] = 0
    elif votacionNegativa == votacionPositiva == 1:
      df_3_1.at[i, 'label_task_3_1_merged'] = 'To_delete' # CONFLICTO. Tenemos 3 opciones etiquetar 0, etiquetar 1 o descartar la fila

# df_3_1

<ipython-input-9-d46e6eb9da8f>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'To_delete' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_3_1.at[i, 'label_task_3_1_merged'] = 'To_delete' # CONFLICTO. Tenemos 3 opciones etiquetar 0, etiquetar 1 o descartar la fila


In [10]:
df_3_1 = df_3_1.drop(columns=["number_annotators","labels_task_3_1_1", "labels_task_3_1_2", "labels_task_3_1_3"])

In [13]:
# Distribución del etiquetado después del tratado inicial de los datos
print(df_3_1['label_task_3_1_merged'].value_counts())

label_task_3_1_merged
0    1306
1    1202
Name: count, dtype: int64


In [12]:
# Eliminación de las filas del dataframe que son conflictivas (2 anotadores con 2 etiquetas distintas)

df_3_1 = df_3_1.drop(df_3_1[df_3_1['label_task_3_1_merged'] == 'To_delete'].index)
df_3_1.reset_index(drop=True, inplace=True)

In [14]:
df_3_1

,lang,text,path_video,label_task_3_1_merged
0,es,cuando ves a las del 08 en la fiesta tu amigo...,videos/7281385962049998086.mp4,1
1,es,mujer sola caminando por la calle | yo automat...,videos/7164058026352168197.mp4,1
2,es,mi amigo no le importa ni las mujeres ni las ...,videos/7248606026386263323.mp4,0
3,es,confirman las chicas cogiendo confianza despué...,videos/7305803156074597665.mp4,1
4,es,aplastar la realidad la gordita del salón tien...,videos/7318400739775204614.mp4,1
...,...,...,...,...
2503,es,temach le contesta a juan guarnizo 'compa_revj...,videos/7200979592717339910.mp4,1
2504,es,"sígueme, persona, rk, nik, meuk, ad0, mefor, ...",videos/7211806336047156486.mp4,0
2505,es,vas a seguir regalando flores? no regales ni a...,videos/7213218074936315141.mp4,1
2506,es,héctor dice que el sábado puse un pedo en cas...,videos/7217265848409492782.mp4,0


In [15]:
# Guardamos el fichero resultante en nuestro Drive
# df_3_1.T.to_csv('/content/drive/MyDrive/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_POSITIVE.csv')
# df_3_1.T.to_csv('/content/drive/MyDrive/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_NEGATIVE.csv')
df_3_1.T.to_csv('/content/drive/MyDrive/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv')

Tratamiento del fichero orifinal para generar uno nuevo en donde solamente se recoja la información relevante para la tarea 3.3

Etiquetas a tratar:
- **IDEOLOGICAL AND INEQUALITY**: el texto desacredita el movimiento feminista, rechaza la desigualdad entre hombres y mujeres o presente a los hombres como víctimas de la opresión del género.
- **STEREOTYPING AND DOMINANCE**: el texto expresa ideas falsas sobre las mujeres que sugieren que son más adecuadas para desempeñar ciertos papeles (madre, esposa, cuidadora familiar, fiel, tierna, cariñosa, sumisa, etc.), o inadecuadas para ciertas tareas (conducir, trabajo duro, etc.), o afirmar que los hombres son de alguna manera superiores a las mujeres.
- **OBJECTIFICATION**: el texto presenta a las mujeres como objetos al margen de su dignidad y aspectos personales, o asume o describe determinadas cualidades físicas que deben tener las mujeres para cumplir los roles tradicionales de género (cumplimiento de cánones de belleza, hipersexualización de los atributos femeninos, cuerpos de las mujeres a disposición de los hombres, etc.).
- **SEXUAL VIOLENCE**: se hacen sugerencias sexuales, se piden favores sexuales o se ejerce el acoso de naturaleza sexual (violación o agresión sexual).
- **MISOGYNY AND NON-SEXUAL VIOLENCE**: el texto expresa odio y violencia hacia las mujeres.

Para esta tarea de clasificación multi-etiqueta, vamos a proponer trabajar con 3 ficheros diferentes (extraídos a partir del fichero original base proporcionado por la competición) los cuáles abarcan los conflictos de 3 maneras diferentes:
- **EXIST2025_training_task3_3_ALL.csv**: el etiquetado final se hará siguiendo el comportamiento de la operación lógica *OR* con el etiquetado de cada uno de los anotadores
- **EXIST2025_training_task3_3_MEN.csv**: En caso de tener conflicto con 2 anotadores prevalecerá la anotación del hombre.
- **EXIST2025_training_task3_3_WOMEN.csv**: En caso de tener conflicto con 2 anotadores prevalecerá la anotación de la mujer.


In [16]:
df = pd.read_json('/content/drive/MyDrive/EXIST 2025 Videos Dataset/training/EXIST2025_training.json', orient='index') # Lectura del dataset desde Drive
df.reset_index(drop=True, inplace=True) # Reseteamos el index del dataframe
df = df.drop(columns=["id_Tiktok", "id_EXIST", "video", "url", "annotators", "labels_task3_2", "split"]) # Eliminación de columnas que no son relevantes del dataframe

In [33]:
# Eliminamos la columna correspondiente a la tarea 3.1
df_3_3 = df.drop(columns=["labels_task3_1"])

# Expande la columna labels_task3_3 y gender_annotators en varias columnas en varias columnas
df_3_3_expanded = df_3_3["labels_task3_3"].apply(pd.Series)
df_3_3_expanded_gender = df_3_3["gender_annotators"].apply(pd.Series)

# Renombra las columnas resultantes para mayor claridad
df_3_3_expanded.columns = [f"labels_task_3_3_{i+1}" for i in range(df_3_3_expanded.shape[1])]
df_3_3_expanded_gender.columns = [f"gender_annotators_{i+1}" for i in range(df_3_3_expanded_gender.shape[1])]

# Une con el dataframe original sin la columna original
df_3_3 = pd.concat([df_3_3.drop(columns=["labels_task3_3"]), df_3_3_expanded], axis=1)
df_3_3 = pd.concat([df_3_3.drop(columns=["gender_annotators"]), df_3_3_expanded_gender], axis=1)

# Reordenamos el orden las columnas para una mejor visión del dataframe
df_3_3 = df_3_3[["lang", "text", "path_video", "number_annotators", "gender_annotators_1", "gender_annotators_2", "gender_annotators_3", "labels_task_3_3_1", "labels_task_3_3_2", "labels_task_3_3_3"]]

# Creación de las nuevas columnas correspondientes a las etiquetas de la tarea 3_3
df_3_3[["IDEOLOGICAL AND INEQUALITY", "STEREOTYPING AND DOMINANCE", "OBJECTIFICATION", "SEXUAL VIOLENCE", "MISOGYNY AND NON-SEXUAL VIOLENCE"]] = 0

# Pasamos a binario los valores YES y NO
# df_3_1.iloc[:, 4:] = df_3_1.iloc[:, 4:].replace({"YES": int(1), "NO": int(0)})

# # # Esto se hace si queremos determinar si un valor específico de una fila/columna de un dataframe es NaN
# # # print(pd.isna(df_3_1.at[0, 'labels_task_3_1_3']))

# df_3_1.iloc[:, 6] = df_3_1.iloc[:, 6].astype("Int64")  # Pasamos los valores binarios de la última columna a int64 --> Tipo entero que soporta NaN

# # print(str(type(df_3_1.iloc[0,6]))=="<class 'pandas._libs.missing.NAType'>")

print(df_3_3['gender_annotators_1'].value_counts(),"\n")
print(df_3_3['gender_annotators_2'].value_counts(), "\n")
print(df_3_3['gender_annotators_3'].value_counts(), "\n")

print(df_3_3['labels_task_3_3_1'].value_counts(), "\n")
print(df_3_3['labels_task_3_3_2'].value_counts(), "\n")
print(df_3_3['labels_task_3_3_3'].value_counts(), "\n")

#df_3_3

gender_annotators_1
F    2524
Name: count, dtype: int64 

gender_annotators_2
M    2032
F     492
Name: count, dtype: int64 

gender_annotators_3
M    428
F    316
Name: count, dtype: int64 

labels_task_3_3_1
[-]                                                                                                                 1326
[STEREOTYPING-DOMINANCE]                                                                                             505
[IDEOLOGICAL-INEQUALITY]                                                                                             243
[OBJECTIFICATION]                                                                                                    144
[UNKNOWN]                                                                                                             53
[SEXUAL-VIOLENCE]                                                                                                     51
[MISOGYNY-NON-SEXUAL-VIOLENCE]                                  